In [3]:
import sys
sys.path.append('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/')
import pandas as pd 
import numpy as np 
import datasets
from datasets import Dataset as TFDataset 
import pickle
from bm25.rank_bm25 import BM25Okapi
import random
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
from transformers import PreTrainedTokenizer, AutoTokenizer
import multiprocessing

In [4]:
def min_max_normalization(score_df, score_name, use_zero=True):
    # score_df_g = cudf.from_pandas(score_df)
    if not use_zero:
        score_df_ = score_df[score_df[score_name] != 0]
    else:
        score_df_ = score_df
    scores_min = score_df_[['sess_id', score_name]].groupby('sess_id')[score_name].min().to_numpy()
    scores_max = score_df_[['sess_id', score_name]].groupby('sess_id')[score_name].max().to_numpy()

    score_df['scores_min'] = scores_min[score_df['sess_id']]
    score_df['scores_max'] = scores_max[score_df['sess_id']]

    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df['min_max_'+score_name] = (score_df[score_name] - score_df['scores_min']) / (score_df['scores_max'] - score_df['scores_min'])
    score_df.drop(columns=['scores_min', 'scores_max'], inplace=True)

    if not use_zero:
        score_df['min_max_'+score_name][score_df[score_name] == 0] = 0.0


    # del scores_sum_g
    # del merged_score_df_g 

In [5]:
def get_rank(score_df:pd.DataFrame, score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df_ = score_df.sort_values(by=['sess_id', score_name], ascending=[True, False]).reset_index(drop=True)
    score_df_[score_name+'_pos'] = score_df_.groupby(by='sess_id')['product'].cumcount().astype('int32')
    score_df_ = score_df_.sort_values(by=['sess_id', 'product']).reset_index(drop=True)
    score_df[score_name+'_pos'] = score_df_[score_name+'_pos']


In [6]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

# Merge valid scores

In [5]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'

In [6]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)


In [7]:
merged_candidates_feature = read_merged_candidates_feature()

In [8]:
rank_score_list = ['all_items_co_graph_count_0', 'all_items_co_graph_count_1', 'all_items_co_graph_count_2']

In [18]:
for score_name in tqdm(rank_score_list):
    score_df = merged_candidates_feature[['sess_id', 'product', score_name]]
    get_rank(score_df, score_name)
    merged_candidates_feature[score_name+'_pos'] = score_df[score_name+'_pos']

  0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_898474/1879186514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[score_name+'_pos'] = score_df_[score_name+'_pos']
 33%|███▎      | 1/3 [02:19<04:39, 139.78s/it]/tmp/ipykernel_898474/1879186514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[score_name+'_pos'] = score_df_[score_name+'_pos']
 67%|██████▋   | 2/3 [04:43<02:22, 142.06s/it]/tmp/ipykernel_898474/1879186514.py:7: SettingWithCopyWarning: 
A value is trying to be set on 

In [21]:
merged_candidates_feature.query('sess_id==10').sort_values(by=['all_items_co_graph_count_0'], ascending=False)[['sess_id', 'all_items_co_graph_count_0_pos', 'all_items_co_graph_count_0']][:15]

,sess_id,all_items_co_graph_count_0_pos,all_items_co_graph_count_0
3271,10,0,42
3054,10,1,39
3260,10,2,35
3292,10,3,33
3099,10,4,27
3098,10,5,23
3060,10,6,20
3347,10,7,18
3302,10,9,17
3057,10,8,17


In [19]:
merged_candidates_feature[['all_items_co_graph_count_0_pos', 'all_items_co_graph_count_0']]

,all_items_co_graph_count_0_pos,all_items_co_graph_count_0
0,156,12
1,204,5
2,157,12
3,205,5
4,271,0
...,...,...
78842194,319,0
78842195,14,31
78842196,320,0
78842197,321,0


In [22]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sess_avg_price,product_price,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,normalized_sasrec_scores_2,...,lyx_avghist_u2i_score,lyx_avghist_i2i_score,lyx_w2v_cos_l1_score,lyx_w2v_cos_l2_score,lyx_w2v_cos_l3_score,normalized_lyx_avghist_u2i_score,normalized_lyx_avghist_i2i_score,all_items_co_graph_count_0_pos,all_items_co_graph_count_1_pos,all_items_co_graph_count_2_pos
0,0,DE,355165591X,0.0,43.256542,8.990000,2.230508,7.658405e-09,0.512931,1.377575e-09,...,7.284950,10.004648,0.328371,0.333689,0.315483,9.865672e-07,2.208272e-05,156,188,152
1,0,DE,3833237058,0.0,43.256542,22.000000,9.605231,1.221631e-05,9.325538,9.255110e-06,...,14.786277,14.609027,0.446276,0.365882,0.374754,1.786123e-03,2.206525e-03,204,209,153
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,0.714114,1.681035e-09,-0.115904,7.345399e-10,...,8.245017,6.915462,0.418034,0.462675,0.440308,2.576786e-06,1.005625e-06,157,152,154
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,8.750996,5.199363e-06,8.507557,4.084482e-06,...,12.711834,10.658710,0.371478,0.351897,0.313646,2.243842e-04,4.247250e-05,205,210,155
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,8.056712,2.596729e-06,5.898870,3.007453e-07,...,11.918351,10.092535,0.456581,0.369536,0.304733,1.014816e-04,2.411134e-05,271,246,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,6.813615,1.076201e-03,7.203015,4.597607e-04,...,14.848063,12.652530,0.643565,0.643565,0.660659,1.258794e-03,4.346779e-04,319,318,318
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,9.030836,9.881445e-03,10.123234,8.526421e-03,...,17.382141,16.312590,0.550828,0.550828,0.559930,1.586686e-02,1.689318e-02,14,17,28
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,0.796892,2.623396e-06,1.711608,1.895152e-06,...,10.297573,9.075425,0.431231,0.431231,0.415055,1.329542e-05,1.215209e-05,320,319,319
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,4.250781,8.296004e-05,6.447586,2.159998e-04,...,12.515525,10.423323,0.460977,0.460977,0.437468,1.221649e-04,4.677731e-05,321,320,320


In [37]:
merged_candidates_feature.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [10]:
merged_candidates_co_graph_rank = merged_candidates_feature[['sess_id', 'product', 'all_items_co_graph_count_0_pos', 'all_items_co_graph_count_1_pos', 'all_items_co_graph_count_2_pos']]

In [12]:
merged_candidates_co_graph_rank.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/transpose/cxl_merged_candidates_co_graph_rank.parquet', engine='pyarrow')

# Merge test scores

In [1]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_test_feature.parquet'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [7]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

In [8]:
merged_candidates_feature_test = read_merged_candidates_feature_test()

In [10]:
rank_score_list = ['all_items_co_graph_count_0', 'all_items_co_graph_count_1', 'all_items_co_graph_count_2']

In [11]:
for score_name in tqdm(rank_score_list):
    score_df = merged_candidates_feature_test[['sess_id', 'product', score_name]]
    get_rank(score_df, score_name)
    merged_candidates_feature_test[score_name+'_pos'] = score_df[score_name+'_pos']

  0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_70767/327185232.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[score_name+'_pos'] = score_df_[score_name+'_pos']
 33%|███▎      | 1/3 [02:16<04:32, 136.18s/it]/tmp/ipykernel_70767/327185232.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[score_name+'_pos'] = score_df_[score_name+'_pos']
 67%|██████▋   | 2/3 [05:13<02:40, 160.24s/it]/tmp/ipykernel_70767/327185232.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [17]:
merged_candidates_feature_test.query('sess_id==100').sort_values(by=['all_items_co_graph_count_1'], ascending=False)[['sess_id', 'all_items_co_graph_count_1_pos', 'all_items_co_graph_count_1']][:15]

,sess_id,all_items_co_graph_count_1_pos,all_items_co_graph_count_1
29743,100,0,95.942398
29713,100,1,93.829765
29714,100,2,87.090996
29715,100,3,83.251221
29717,100,4,55.586777
29684,100,5,53.190960
29683,100,6,50.660015
29748,100,7,49.570526
29616,100,8,38.484016
29652,100,9,36.483471


In [14]:
merged_candidates_feature_test

,sess_id,sess_locale,product,sasrec_scores_2,normalized_sasrec_scores_2,sasrec_scores_3,normalized_sasrec_scores_3,sess_avg_price,product_price,seqmlp_scores,...,last2item_freq_test,feat_BM25_scores,min_max_feat_BM25_scores,min_max_title_BM25_scores,min_max_desc_BM25_scores,min_max_title_bert_scores,min_max_text_bert_scores,all_items_co_graph_count_0_pos,all_items_co_graph_count_1_pos,all_items_co_graph_count_2_pos
0,0,DE,B000Q87D0Q,0.000000,3.282997e-10,0.000000,6.689660e-10,67.527199,36.761604,0.000000,...,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,101,114,59
1,0,DE,B000QB30DW,0.501346,5.420036e-10,-0.588501,3.713825e-10,67.527199,9.990000,7.260942,...,71,0.000000,0.000000,0.000000,0.010312,0.726431,0.826050,71,78,60
2,0,DE,B004BIG55Q,6.917523,3.315223e-07,5.737720,2.076175e-07,67.527199,8.990000,2.454817,...,346,9.070796,0.087639,0.000000,0.062083,0.531170,0.683499,90,111,61
3,0,DE,B0053FTNQY,-0.100895,2.967921e-10,1.507319,3.020121e-09,67.527199,36.761604,3.837643,...,11,1.598722,0.015446,0.000000,0.050930,0.350456,0.473276,127,118,62
4,0,DE,B007QWII1S,3.768980,1.422714e-08,4.594047,6.615662e-08,67.527199,54.950001,4.923371,...,27,50.413322,0.487077,0.309716,0.204088,0.676267,0.715921,194,156,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96556030,316971,UK,B0B82N3CQQ,-1.076433,6.007382e-08,-0.457645,1.105378e-07,19.459999,13.990000,6.433315,...,3,0.000000,0.000000,0.372690,0.114051,0.694524,0.771414,318,317,316
96556031,316971,UK,B0BB9NW3F3,0.000000,1.762683e-07,0.000000,1.746882e-07,19.459999,22.097065,0.000000,...,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,319,318,317
96556032,316971,UK,B0BDMVKTQ3,-1.079334,5.989980e-08,-1.901198,2.609658e-08,19.459999,41.990002,-1.094359,...,24,0.000000,0.000000,0.000000,0.000000,0.175180,0.211611,171,75,318
96556033,316971,UK,B0BHW1D5VP,6.722834,1.465088e-04,6.111193,7.876277e-05,19.459999,26.990000,8.700006,...,10,19.868626,0.115995,0.265195,0.050508,0.610117,0.629789,172,319,319


In [15]:
merged_candidates_co_graph_rank_test = merged_candidates_feature_test[['sess_id', 'product', 'all_items_co_graph_count_0_pos', 'all_items_co_graph_count_1_pos', 'all_items_co_graph_count_2_pos']]

In [16]:
merged_candidates_co_graph_rank_test.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/transpose/cxl_merged_candidates_co_graph_rank_test.parquet', engine='pyarrow')

In [18]:
merged_candidates_co_graph_rank_test

,sess_id,product,all_items_co_graph_count_0_pos,all_items_co_graph_count_1_pos,all_items_co_graph_count_2_pos
0,0,B000Q87D0Q,101,114,59
1,0,B000QB30DW,71,78,60
2,0,B004BIG55Q,90,111,61
3,0,B0053FTNQY,127,118,62
4,0,B007QWII1S,194,156,63
...,...,...,...,...,...
96556030,316971,B0B82N3CQQ,318,317,316
96556031,316971,B0BB9NW3F3,319,318,317
96556032,316971,B0BDMVKTQ3,171,75,318
96556033,316971,B0BHW1D5VP,172,319,319
